In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from icepy4d.least_squares import rototra3d

from icepy4d.utils.transformations import Rotrotranslation

icepy_path = Path.cwd().parents[1]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Create the Rototranslation object by passing a 3D transformation matrix to constrution, as: 

```
rotra = Rotrotranslation(Tmat)
```

The 3D transformation matrix is a x4 numpy array expressed in homogeneous coordinates (last row is [0, 0, 0, 1]) and must contain both the rotion matrix and the traslation vector, as:

```
Tmat =  r11 | r12 | r13 | tx
        r21 | r22 | r23 | ty
        r31 | r32 | r33 | tz
        0   | 0   | 0   | 1
```

For the Belvedere Glacier, the transformation between the carthesian local reference system and the WGS84 UTM32N reference system are stored within the Rototranslation class. 
Therefore, the Rototranslation object can be directly initialized as:

```
rotra_loc2UTM = Rotrotranslation.belvedere_loc2utm()
rotra_UTM2loc = Rotrotranslation.belvedere_utm2loc()

```


In [4]:
belv_rotra = Rotrotranslation.belvedere_loc2utm()
print(belv_rotra.T)

[[ 7.06579328e-01 -7.06873715e-01 -1.26001140e-04  4.16614833e+05]
 [ 7.06873715e-01  7.06579268e-01  2.02054813e-04  5.09093271e+06]
 [-5.38263700e-05 -2.31959390e-04  9.99462247e-01  1.76754700e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


After having initialized the rototranslation object with the transformation matrix, the transformation can be applied to a point or a list of points.
To this end, let's first read the series of point from a text file by using pandas and save the coordinates in a numpy array with shape (3,N).

In [8]:
fname = "res/DIC3D/movimento_DIC_3D.txt"
col_names = ["X", "Y", "Z", "dX", "dY", "dZ", "C"]
points = pd.read_csv(icepy_path / fname, header=None, names=col_names)
xyz = points[["X", "Y", "Z"]].to_numpy().T
print(xyz.shape)

(3, 4980)


Apply the transformation to the points by using the `apply_transformation` method of the Rototranslation class:

In [ ]:
enh = belv_rotra.apply_transformation(xyz).T

In [ ]:
col_names = ["E", "N", "h", "dE", "dN", "dh", "C"]
points_utm = pd.DataFrame(columns=col_names)
points_utm[["E", "N", "h"]] = enh
points_utm["C"] = points["C"]

In [ ]:
xyz = points[["X", "Y", "Z"]].to_numpy()
dxyz = points[["dX", "dY", "dZ"]].to_numpy()
xyz_fin = xyz + dxyz

enh_fin = belv_rotra.apply_transformation(xyz_fin.T).T
points_utm[["E_fin", "N_fin", "h_fin"]] = enh

dENh = enh_fin - enh
points_utm[["dE", "dN", "dh"]] = dENh

In [ ]:
fname = "res/DIC3D/movimento_DIC_3D_UTM.txt"
points_utm.to_csv(icepy_path / fname, sep=",", index=False)

In [ ]:
fname = "data/targets/target_world.csv"
out_name = "data/targets/target_utm.csv"

target_loc= pd.read_csv(icepy_path / fname)

targets_utm = pd.DataFrame(columns=["label", "E", "N", "h"])
targets_utm["label"] = target_loc["label"]
targets_utm[['E', 'N', 'h']] = belv_rotra.apply_transformation(target_loc[["X", "Y", "Z"]].to_numpy().T).T
targets_utm.to_csv(icepy_path / out_name, sep=",", index=False)